## Our Model
Here we will be creating different models to classify our data. 

In [54]:
import numpy as np
from numpy import array
from numpy import array
from numpy import asarray
from numpy import zeros
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# set configurations
pd.set_option('display.max_columns', 100)
sns.set_style("white")

# keras imports
import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

# model imports
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
import pickle
import joblib

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


In [55]:
model_data = pd.read_csv('../data/scheme1.csv', keep_default_na=False)

In [56]:
model_data.head(3)

,Unnamed: 0,Unnamed: 0.1,title,selftext,author,num_comments,is_suicide,url,selftext_clean,title_clean,author_clean,selftext_length,title_length,megatext_clean,Clustered Labels,New Labels
0,0,0,"Our most-broken and least-understood rules is ""helpers may not invite private contact as a first resort"", so we've made a new wiki to explain it","We understand that most people who reply immediately to an OP with an invitation to talk privately mean only to help, but this type of response usually leads to either disappointment or disaster....",SQLwitch,133,0,https://www.reddit.com/r/depression/comments/doqwow/our_mostbroken_and_leastunderstood_rules_is/,understand people reply immediately op invitation talk privately mean help type response usually lead either disappointment disaster usually work quite differently say pm anytime casual social con...,broken least understood rule helper may invite private contact first resort made new wiki explain,sql witch,4792,144,sql witch understand people reply immediately op invitation talk privately mean help type response usually lead either disappointment disaster usually work quite differently say pm anytime casual ...,1,1
1,1,1,Regular Check-In Post,Welcome to /r/depression's check-in post - a place to take a moment and share what is going on and how you are doing. If you have an accomplishment you want to talk about (these shouldn't be stand...,circinia,1644,0,https://www.reddit.com/r/depression/comments/exo6f1/regular_checkin_post/,welcome r depression check post place take moment share going accomplishment want talk standalone post sub violate role model rule welcome tough time prefer make post place share try best keep spa...,regular check post,c irc,650,21,c irc welcome r depression check post place take moment share going accomplishment want talk standalone post sub violate role model rule welcome tough time prefer make post place share try best ke...,1,1
2,2,2,"I hate it so much when you try and express your feelings to your parents, but they turn it around and compare your suffering with theirs.","I've been feeling really depressed and lonely lately from my job, I'm a full time late night janitor for a courthouse just 10 miles away from my hometown. Working there has been pretty easy, but I...",TheNewKiller69,8,0,https://www.reddit.com/r/depression/comments/fedwbi/i_hate_it_so_much_when_you_try_and_express_your/,feeling really depressed lonely lately job full time late night janitor courthouse 10 mile away hometown working ha pretty easy wound feeling super lonely lot considering really get talk friend ha...,hate much try express feeling parent turn around compare suffering,new killer 69,1866,137,new killer 69 feeling really depressed lonely lately job full time late night janitor courthouse 10 mile away hometown working ha pretty easy wound feeling super lonely lot considering really get ...,0,0


In [57]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1897 entries, 0 to 1896
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1897 non-null   int64 
 1   Unnamed: 0.1      1897 non-null   int64 
 2   title             1897 non-null   object
 3   selftext          1897 non-null   object
 4   author            1897 non-null   object
 5   num_comments      1897 non-null   int64 
 6   is_suicide        1897 non-null   int64 
 7   url               1897 non-null   object
 8   selftext_clean    1897 non-null   object
 9   title_clean       1897 non-null   object
 10  author_clean      1897 non-null   object
 11  selftext_length   1897 non-null   int64 
 12  title_length      1897 non-null   int64 
 13  megatext_clean    1897 non-null   object
 14  Clustered Labels  1897 non-null   int64 
 15  New Labels        1897 non-null   int64 
dtypes: int64(8), object(8)
memory usage: 237.2+ KB


Establishing a baseline accuracy is important for evaluating the model's progression. If every prediction was 1, let's see what our accuracy would be.

In [58]:
model_data['is_suicide'].mean()

0.5166051660516605

Our baseline accuracy is about 51.7%

### Selection of Features and Attributes
Here we are attempting to create a model using CountVectorizer and Naive Bayes Model to determine which columns are the best to score and use for our model.

In [59]:
# Creating a function to score different models from different columsn in the dataset

columns_list = ['column_1', "column_2", "column_3"]
model = "CountVec + MultinomialNB"
df_list=[] 

def multi_modelling(columns_list, model):
    for i in columns_list:
        
        X = model_data[i]
        y = model_data['is_suicide']
        
        # train/test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
        
        cvec = CountVectorizer()
        cvec.fit(X_train)
        
        # creating dataframes
        X_train = pd.DataFrame(cvec.transform(X_train).todense(),
                               columns=cvec.get_feature_names())
        X_test = pd.DataFrame(cvec.transform(X_test).todense(),
                               columns=cvec.get_feature_names())
        
        nb = MultinomialNB()
        nb.fit(X_train,y_train)
        
        # get predictions from model
        pred = nb.predict(X_test)
        
        # create confusion matrix
        tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
        
        # calculating AUC
        nb.predict_proba(X_test)
        pred_proba = [i[1] for i in nb.predict_proba(X_test)] 
        auc = roc_auc_score(y_test, pred_proba)

        # classification report
        classi_dict = (classification_report(y_test,pred, output_dict=True))

        # storing our results
        model_results = {}
        model_results['series used (X)'] = i
        model_results['model'] = model
        model_results['AUC Score'] = auc
        model_results['precision']= classi_dict['weighted avg']['precision']
        model_results['recall (sensitivity)']= classi_dict['weighted avg']['recall']
        model_results['confusion matrix']={"TP": tp,"FP":fp, "TN": tn, "FN": fn}
        model_results['train accuracy'] = nb.score(X_train, y_train)
        model_results['test accuracy'] = nb.score(X_test, y_test)
        model_results['baseline accuracy']=0.5166
        model_results['specificity']= tn/(tn+fp)  
        model_results['f1-score']= classi_dict['weighted avg']['f1-score']
        #model_results['support']= classi_dict['weighted avg']['support']
        model_results
        df_list.append(model_results) 

    pd.set_option("display.max_colwidth", 50)
    return (pd.DataFrame(df_list)).round(2)

For our initial model, it will be a binary classifier. Once the user gets their classification, they will go to another model that will give them specific support. 

Label Encoding: r/SuicideWatch = 1, r/Depression = 0

TP: model predicts suicide, and it is correct

TN: model predicts depression, and it is correct

FP: model predicts suicide, but it is really depression, not good

FN: model predicts depression, but they really are suicidal, this is the worst, misses an at risk patient

In [60]:
# using the function
columns_list = ['selftext', "author", "title",'selftext_clean', "author_clean", "title_clean", "megatext_clean"]
model = "CountVec + MultinomialNB"
df_list=[]
multi_modelling(columns_list, model)

,series used (X),model,AUC Score,precision,recall (sensitivity),confusion matrix,train accuracy,test accuracy,baseline accuracy,specificity,f1-score
0,selftext,CountVec + MultinomialNB,0.69,0.66,0.66,"{'TP': 161, 'FP': 78, 'TN': 152, 'FN': 84}",0.92,0.66,0.52,0.66,0.66
1,author,CountVec + MultinomialNB,0.57,0.63,0.55,"{'TP': 235, 'FP': 204, 'TN': 26, 'FN': 10}",0.99,0.55,0.52,0.11,0.45
2,title,CountVec + MultinomialNB,0.67,0.62,0.62,"{'TP': 167, 'FP': 104, 'TN': 126, 'FN': 78}",0.85,0.62,0.52,0.55,0.62
3,selftext_clean,CountVec + MultinomialNB,0.69,0.67,0.67,"{'TP': 165, 'FP': 78, 'TN': 152, 'FN': 80}",0.91,0.67,0.52,0.66,0.67
4,author_clean,CountVec + MultinomialNB,0.54,0.51,0.51,"{'TP': 169, 'FP': 155, 'TN': 75, 'FN': 76}",0.95,0.51,0.52,0.33,0.50
5,title_clean,CountVec + MultinomialNB,0.67,0.63,0.62,"{'TP': 178, 'FP': 112, 'TN': 118, 'FN': 67}",0.84,0.62,0.52,0.51,0.62
6,megatext_clean,CountVec + MultinomialNB,0.71,0.67,0.67,"{'TP': 160, 'FP': 71, 'TN': 159, 'FN': 85}",0.95,0.67,0.52,0.69,0.67


Based on our results, using the megatext_clean column will be the best option for our model due its good metrics, such as its AUC value. The false negatives are the highest in our confusion matrix, meaning it is good for the model to learn from. 

### Production Model
We will test a few different models by running some perumatations and recording the data.

In [61]:
# Defining a function that test different models and record parameters

def gridsearch_multi(steps_titles, steps_list, pipe_params):
    
    X = model_data["megatext_clean"]
    y = model_data['is_suicide']
    
    # train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    # build dataframe
    gs_results = pd.DataFrame(columns=['model','AUC Score', 'precision', 'recall (sensitivity)', 
                                       'best_params', 'best score', 'confusion matrix', 
                                       'train_accuracy','test_accuracy','baseline_accuracy',
                                       'specificity', 'f1-score'])

    for i in range(len(steps_list)):
        # define pipeline
        pipe = Pipeline(steps=steps_list[i])

        gs = GridSearchCV(pipe, pipe_params[i], cv=3) 
        gs.fit(X_train, y_train)
        
        # getting predictions from model
        pred = gs.predict(X_test)
        
        # confusion matrix
        tn, fp, fn, tp = confusion_matrix(y_test, gs.predict(X_test)).ravel() 
        
        classi_dict = (classification_report(y_test,pred, output_dict=True))
        
        # calculating AUC
        gs.predict_proba(X_test)
        pred_proba = [i[1] for i in gs.predict_proba(X_test)] 
        auc = roc_auc_score(y_test, pred_proba)
        
        # create dataframe columns
        model_results = {}
        model_results['model'] = steps_titles[i]
        model_results['AUC Score'] = auc
        model_results['precision']= classi_dict['weighted avg']['precision']
        model_results['recall (sensitivity)']= classi_dict['weighted avg']['recall']
        model_results['best params'] = gs.best_params_
        model_results['best score'] = gs.best_score_
        model_results['confusion matrix']={"TP": tp,"FP":fp, "TN": tn, "FN": fn}
        model_results['train accuracy'] = gs.score(X_train, y_train)
        model_results['test accuracy'] = gs.score(X_test, y_test)
        model_results['baseline accuracy'] = 0.5166
        
        model_results['specificity']= tn/(tn+fp)  
        model_results['f1-score']= classi_dict['weighted avg']['f1-score']


        # add data
        df_list.append(model_results) 
        pd.set_option("display.max_colwidth", 200)
    return (pd.DataFrame(df_list)).round(2)


In [62]:
# testing function with count vectorizor

df_list=[]

# models
steps_titles = ['cvec+ multi_nb','cvec + ss + knn','cvec + ss + logreg']

# pipelines
steps_list = [ 
    [('cv', CountVectorizer()),('multi_nb', MultinomialNB())],
    [('cv', CountVectorizer()),('scaler', StandardScaler(with_mean=False)),('knn', KNeighborsClassifier())], 
    [('cv', CountVectorizer()),('scaler', StandardScaler(with_mean=False)),('logreg', LogisticRegression())]
]

# parameters
pipe_params = [
    {'cv__stop_words':['english'], 'cv__ngram_range':[(1,1),(1,2)],'cv__max_features': [20, 30, 50],'cv__min_df': [2, 3],'cv__max_df': [.2, .25, .3]},
    {'cv__stop_words':['english'], 'cv__ngram_range':[(1,1),(1,2)],'cv__max_features': [20, 30, 50],'cv__min_df': [2, 3],'cv__max_df': [.2, .25, .3]},
    {'cv__stop_words':['english'], 'cv__ngram_range':[(1,1),(1,2)],'cv__max_features': [20, 30, 50],'cv__min_df': [2, 3],'cv__max_df': [.2, .25, .3]}
]   

gridsearch_multi(steps_titles, steps_list, pipe_params)

,model,AUC Score,precision,recall (sensitivity),best params,best score,confusion matrix,train accuracy,test accuracy,baseline accuracy,specificity,f1-score
0,cvec+ multi_nb,0.72,0.67,0.67,"{'cv__max_df': 0.3, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.65,"{'TP': 160, 'FP': 71, 'TN': 159, 'FN': 85}",0.68,0.67,0.52,0.69,0.67
1,cvec + ss + knn,0.60,0.58,0.58,"{'cv__max_df': 0.2, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.60,"{'TP': 144, 'FP': 100, 'TN': 130, 'FN': 101}",0.72,0.58,0.52,0.57,0.58
2,cvec + ss + logreg,0.73,0.69,0.69,"{'cv__max_df': 0.3, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.65,"{'TP': 173, 'FP': 75, 'TN': 155, 'FN': 72}",0.69,0.69,0.52,0.67,0.69


In [63]:
# testing function with TFD Vectorizor

# models
steps_titles = ['tvec + multi_nb','tvec + ss + knn','tvec + ss + logreg']

# pipelines
steps_list = [ 
    [('tv', TfidfVectorizer()),('multi_nb', MultinomialNB())],
    [('tv', TfidfVectorizer()),('scaler', StandardScaler(with_mean=False)),('knn', KNeighborsClassifier())], 
    [('tv', TfidfVectorizer()),('scaler', StandardScaler(with_mean=False)),('logreg', LogisticRegression())]
]

# parameters
pipe_params = [
    {'tv__stop_words':['english'], 'tv__ngram_range':[(1,1),(1,2)],'tv__max_features': [20, 30, 50],'tv__min_df': [2, 3],'tv__max_df': [.2, .25, .3]},
    {'tv__stop_words':['english'], 'tv__ngram_range':[(1,1),(1,2)],'tv__max_features': [20, 30, 50],'tv__min_df': [2, 3],'tv__max_df': [.2, .25, .3]},
    {'tv__stop_words':['english'], 'tv__ngram_range':[(1,1),(1,2)],'tv__max_features': [20, 30, 50],'tv__min_df': [2, 3],'tv__max_df': [.2, .25, .3]}
]   

gridsearch_multi(steps_titles, steps_list, pipe_params)

,model,AUC Score,precision,recall (sensitivity),best params,best score,confusion matrix,train accuracy,test accuracy,baseline accuracy,specificity,f1-score
0,cvec+ multi_nb,0.72,0.67,0.67,"{'cv__max_df': 0.3, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.65,"{'TP': 160, 'FP': 71, 'TN': 159, 'FN': 85}",0.68,0.67,0.52,0.69,0.67
1,cvec + ss + knn,0.60,0.58,0.58,"{'cv__max_df': 0.2, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.60,"{'TP': 144, 'FP': 100, 'TN': 130, 'FN': 101}",0.72,0.58,0.52,0.57,0.58
2,cvec + ss + logreg,0.73,0.69,0.69,"{'cv__max_df': 0.3, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.65,"{'TP': 173, 'FP': 75, 'TN': 155, 'FN': 72}",0.69,0.69,0.52,0.67,0.69
3,tvec + multi_nb,0.73,0.68,0.68,"{'tv__max_df': 0.3, 'tv__max_features': 50, 'tv__min_df': 2, 'tv__ngram_range': (1, 2), 'tv__stop_words': 'english'}",0.65,"{'TP': 169, 'FP': 77, 'TN': 153, 'FN': 76}",0.68,0.68,0.52,0.67,0.68
4,tvec + ss + knn,0.56,0.54,0.54,"{'tv__max_df': 0.2, 'tv__max_features': 50, 'tv__min_df': 2, 'tv__ngram_range': (1, 1), 'tv__stop_words': 'english'}",0.60,"{'TP': 146, 'FP': 118, 'TN': 112, 'FN': 99}",0.74,0.54,0.52,0.49,0.54
5,tvec + ss + logreg,0.73,0.67,0.67,"{'tv__max_df': 0.3, 'tv__max_features': 50, 'tv__min_df': 2, 'tv__ngram_range': (1, 2), 'tv__stop_words': 'english'}",0.65,"{'TP': 160, 'FP': 72, 'TN': 158, 'FN': 85}",0.68,0.67,0.52,0.69,0.67


In [64]:
# testing function with Hashing Vectorizor

# models
steps_titles = ['hvec + multi_nb','hvec + ss + knn','hvec + ss + logreg']

# pipelines
steps_list = [ 
    [('hv', HashingVectorizer(alternate_sign=False)),('multi_nb', MultinomialNB())],
    [('hv', HashingVectorizer(alternate_sign=False)),('scaler', StandardScaler(with_mean=False)),('knn', KNeighborsClassifier())], 
    [('hv', HashingVectorizer(alternate_sign=False)),('scaler', StandardScaler(with_mean=False)),('logreg', LogisticRegression())]
]

# parameters
pipe_params = [
    {'hv__stop_words':['english'], 'hv__ngram_range':[(1,1),(1,2)]},
    {'hv__stop_words':['english'], 'hv__ngram_range':[(1,1),(1,2)]},
    {'hv__stop_words':['english'], 'hv__ngram_range':[(1,1),(1,2)]}
]   

gridsearch_multi(steps_titles, steps_list, pipe_params)


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,model,AUC Score,precision,recall (sensitivity),best params,best score,confusion matrix,train accuracy,test accuracy,baseline accuracy,specificity,f1-score
0,cvec+ multi_nb,0.72,0.67,0.67,"{'cv__max_df': 0.3, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.65,"{'TP': 160, 'FP': 71, 'TN': 159, 'FN': 85}",0.68,0.67,0.52,0.69,0.67
1,cvec + ss + knn,0.60,0.58,0.58,"{'cv__max_df': 0.2, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.60,"{'TP': 144, 'FP': 100, 'TN': 130, 'FN': 101}",0.72,0.58,0.52,0.57,0.58
2,cvec + ss + logreg,0.73,0.69,0.69,"{'cv__max_df': 0.3, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.65,"{'TP': 173, 'FP': 75, 'TN': 155, 'FN': 72}",0.69,0.69,0.52,0.67,0.69
3,tvec + multi_nb,0.73,0.68,0.68,"{'tv__max_df': 0.3, 'tv__max_features': 50, 'tv__min_df': 2, 'tv__ngram_range': (1, 2), 'tv__stop_words': 'english'}",0.65,"{'TP': 169, 'FP': 77, 'TN': 153, 'FN': 76}",0.68,0.68,0.52,0.67,0.68
4,tvec + ss + knn,0.56,0.54,0.54,"{'tv__max_df': 0.2, 'tv__max_features': 50, 'tv__min_df': 2, 'tv__ngram_range': (1, 1), 'tv__stop_words': 'english'}",0.60,"{'TP': 146, 'FP': 118, 'TN': 112, 'FN': 99}",0.74,0.54,0.52,0.49,0.54
5,tvec + ss + logreg,0.73,0.67,0.67,"{'tv__max_df': 0.3, 'tv__max_features': 50, 'tv__min_df': 2, 'tv__ngram_range': (1, 2), 'tv__stop_words': 'english'}",0.65,"{'TP': 160, 'FP': 72, 'TN': 158, 'FN': 85}",0.68,0.67,0.52,0.69,0.67
6,hvec + multi_nb,0.77,0.69,0.68,"{'hv__ngram_range': (1, 1), 'hv__stop_words': 'english'}",0.72,"{'TP': 148, 'FP': 54, 'TN': 176, 'FN': 97}",0.89,0.68,0.52,0.77,0.68
7,hvec + ss + knn,0.51,0.75,0.52,"{'hv__ngram_range': (1, 1), 'hv__stop_words': 'english'}",0.52,"{'TP': 245, 'FP': 229, 'TN': 1, 'FN': 0}",0.52,0.52,0.52,0.00,0.36
8,hvec + ss + logreg,0.65,0.62,0.62,"{'hv__ngram_range': (1, 1), 'hv__stop_words': 'english'}",0.63,"{'TP': 155, 'FP': 90, 'TN': 140, 'FN': 90}",1.00,0.62,0.52,0.61,0.62


After reviewing all the performances, two models stand out. The Hashing Vectorizor and Multinomial Naive Bayes model had much better metrics, along with the TFID Vectorizor and Multinomial Naive Bayes Model. They both have good AUC values and high generalization.

### Tuning Hyperparameters

In [65]:
# using the function for out two narrowed models

# models
steps_titles = ['hvec + multi_nb(tuning)','tvec + multi_nb(tuning)']

# pipelines
steps_list = [ 
    [('hv', HashingVectorizer(alternate_sign=False)),('multi_nb', MultinomialNB())],
    [('tv', TfidfVectorizer()),('multi_nb', MultinomialNB())]
]

# parameters
pipe_params = [
    {'hv__stop_words':['english'], 'hv__ngram_range':[(1,1),(1,2),(1,3)], 'hv__n_features': [50, 150, 300, 500, 800, 1000]},
    {'tv__stop_words':['english'], 'tv__ngram_range':[(1,1),(1,2),(1,3)],'tv__max_features': [20, 30, 50, 70, 100],'tv__min_df': [2, 3],'tv__max_df': [.2, .25, .3, .35,.4]},
]   

gridsearch_multi(steps_titles, steps_list, pipe_params)

,model,AUC Score,precision,recall (sensitivity),best params,best score,confusion matrix,train accuracy,test accuracy,baseline accuracy,specificity,f1-score
0,cvec+ multi_nb,0.72,0.67,0.67,"{'cv__max_df': 0.3, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.65,"{'TP': 160, 'FP': 71, 'TN': 159, 'FN': 85}",0.68,0.67,0.52,0.69,0.67
1,cvec + ss + knn,0.60,0.58,0.58,"{'cv__max_df': 0.2, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.60,"{'TP': 144, 'FP': 100, 'TN': 130, 'FN': 101}",0.72,0.58,0.52,0.57,0.58
2,cvec + ss + logreg,0.73,0.69,0.69,"{'cv__max_df': 0.3, 'cv__max_features': 50, 'cv__min_df': 2, 'cv__ngram_range': (1, 1), 'cv__stop_words': 'english'}",0.65,"{'TP': 173, 'FP': 75, 'TN': 155, 'FN': 72}",0.69,0.69,0.52,0.67,0.69
3,tvec + multi_nb,0.73,0.68,0.68,"{'tv__max_df': 0.3, 'tv__max_features': 50, 'tv__min_df': 2, 'tv__ngram_range': (1, 2), 'tv__stop_words': 'english'}",0.65,"{'TP': 169, 'FP': 77, 'TN': 153, 'FN': 76}",0.68,0.68,0.52,0.67,0.68
4,tvec + ss + knn,0.56,0.54,0.54,"{'tv__max_df': 0.2, 'tv__max_features': 50, 'tv__min_df': 2, 'tv__ngram_range': (1, 1), 'tv__stop_words': 'english'}",0.60,"{'TP': 146, 'FP': 118, 'TN': 112, 'FN': 99}",0.74,0.54,0.52,0.49,0.54
5,tvec + ss + logreg,0.73,0.67,0.67,"{'tv__max_df': 0.3, 'tv__max_features': 50, 'tv__min_df': 2, 'tv__ngram_range': (1, 2), 'tv__stop_words': 'english'}",0.65,"{'TP': 160, 'FP': 72, 'TN': 158, 'FN': 85}",0.68,0.67,0.52,0.69,0.67
6,hvec + multi_nb,0.77,0.69,0.68,"{'hv__ngram_range': (1, 1), 'hv__stop_words': 'english'}",0.72,"{'TP': 148, 'FP': 54, 'TN': 176, 'FN': 97}",0.89,0.68,0.52,0.77,0.68
7,hvec + ss + knn,0.51,0.75,0.52,"{'hv__ngram_range': (1, 1), 'hv__stop_words': 'english'}",0.52,"{'TP': 245, 'FP': 229, 'TN': 1, 'FN': 0}",0.52,0.52,0.52,0.00,0.36
8,hvec + ss + logreg,0.65,0.62,0.62,"{'hv__ngram_range': (1, 1), 'hv__stop_words': 'english'}",0.63,"{'TP': 155, 'FP': 90, 'TN': 140, 'FN': 90}",1.00,0.62,0.52,0.61,0.62
9,hvec + multi_nb(tuning),0.75,0.68,0.68,"{'hv__n_features': 1000, 'hv__ngram_range': (1, 1), 'hv__stop_words': 'english'}",0.69,"{'TP': 170, 'FP': 75, 'TN': 155, 'FN': 75}",0.82,0.68,0.52,0.67,0.68


### TF-IDF Vectorizor + Multinomial Naive Bayes
This model has the best metrics and will work the best for our data. While it isn't the higher AUC, it generalizes better and has higher True Positives. 

## Running our Optimized Model
This model is a combination of TF-IDF("Term Frequency - Inverse Document" Frequency) Vectorizer and the Multinomial Naive Bayes. It assigns scores for the top 70 words in our selected feature. TF-IDF will penalize common words, helping the model find specific key words. The model makes a prediction based on a matrix of word scores and gives a probability of falling into a certain classification. 

In [72]:
# getting ready for training

X = model_data["selftext_clean"]
y = model_data["is_suicide"]
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

tvec_optimised = TfidfVectorizer(max_df= 0.5, max_features=70, min_df=2, ngram_range=(1, 3),stop_words = 'english')
X_train_tvec = tvec_optimised.fit_transform(X_train).todense()
X_test_tvec = tvec_optimised.transform(X_test).todense()

In [76]:
from keras.preprocessing import sequence

vocabulary_size = 5000
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

ValueError: invalid literal for int() with base 10: ' want loose dont know healthy keep seeing hanging mean kinda depressed bad feeling thought come go also worried distance would intensify insecurites love another guy probalby stop seeing tho least to

In [45]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size= 32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

batch_size = 64
num_epochs = 10
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None
Train on 1453 samples, validate on 64 samples
Epoch 1/10
1453/1453 [==============================] - 12s 8ms/step - loss: 0.6847 - accuracy: 0.5272 - val_loss: 0.6629 - val_accuracy: 0.6562
Epoch 2/10
1453/1453 [==============================] - 11s 7ms/step - loss: 0.6666 - accuracy: 0.5843 - val_loss: 0.6096 - val_accuracy: 0.6875
Epoch 3/10
1453/1453 [==============

In [49]:
scores = model.evaluate(X_valid, y_valid, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.765625


In [77]:
model.save("RNN.h5")

AttributeError: 'str' object has no attribute 'save'

In [16]:
embeddings_dictionary = dict()

embedding_matrix = zeros((vocab_size, 70))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [17]:
model = Sequential()

embedding_layer = Embedding(vocab_size, 70, weights=[embedding_matrix], input_length=70 , trainable=False)
model.add(embedding_layer)

model.add(keras.layers.Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [18]:
model1 = Sequential()
model1.add(Dense(10, input_dim=70, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [19]:
model2 = Sequential()

model2.add(Flatten())
model2.add(Dense(10, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D

model3 = Sequential()
model3.add(Dense(1, activation='sigmoid'))

model3.add(Dense(10, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [42]:
from keras.layers.convolutional import MaxPooling1D

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length= 70))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 70, 100)           888100    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 63, 32)            25632     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 31, 32)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 992)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                9930      
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 11        
Total params: 923,673
Trainable params: 923,673
Non-trainable params: 0
_______________________________________________

In [43]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
history = model.fit(X_train_tvec, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(X_test_tvec, y_test))

Train on 1517 samples, validate on 380 samples
Epoch 1/100
1517/1517 [==============================] - 1s 915us/step - loss: 0.6947 - accuracy: 0.5063 - val_loss: 0.6930 - val_accuracy: 0.5158
Epoch 2/100
1517/1517 [==============================] - 1s 683us/step - loss: 0.6929 - accuracy: 0.5168 - val_loss: 0.6928 - val_accuracy: 0.5158
Epoch 3/100
1517/1517 [==============================] - 1s 568us/step - loss: 0.6932 - accuracy: 0.5023 - val_loss: 0.6927 - val_accuracy: 0.5158
Epoch 4/100
1517/1517 [==============================] - 1s 651us/step - loss: 0.6929 - accuracy: 0.5168 - val_loss: 0.6928 - val_accuracy: 0.5158
Epoch 5/100
1517/1517 [==============================] - 1s 640us/step - loss: 0.6928 - accuracy: 0.5168 - val_loss: 0.6927 - val_accuracy: 0.5158
Epoch 6/100
1517/1517 [==============================] - 1s 602us/step - loss: 0.6930 - accuracy: 0.5168 - val_loss: 0.6927 - val_accuracy: 0.5158
Epoch 7/100
1517/1517 [==============================] - 1s 543us/step 

KeyboardInterrupt: 

### Model Performace
